# Cluster

In [8]:
import sys; sys.path.append('../..')
from abslithist.words import *
from abslithist.models import *

In [9]:
fields = get_allfields()

In [10]:
words = list(fields['Abs-Conc.Median.Conc.median'])
len(words)

23776

In [15]:
ofn_abs='data.top_words.abs.json'
gen_top_words_across_models(
    fields['Abs-Conc.Median.Abs.median'],
    ofn=ofn_abs,
    num_proc=4
)

In [ ]:
stop

In [19]:
df_abs=pd.DataFrame(readgen_jsonl('../../data/dists/data.top_words.abs.json'))
df_abs['absconc']='Abs'
df_conc=pd.DataFrame(readgen_jsonl('../../data/dists/data.top_words.json'))
df_conc['absconc']='Conc'
df=pd.concat([df_abs,df_conc])

In [20]:
df

,word,neighborhood,corpus,period_start,period_end,run,absconc
0,ineffable,"[[benignity, 0.7806432843208313], [undissemble...",coha,1800,1900,run_07,Abs
1,enterprizes,"[[rivalship, 0.8295625448226929], [fellowmen, ...",coha,1800,1900,run_07,Abs
2,harbored,"[[molested, 0.6936917304992676], [taunted, 0.6...",coha,1800,1900,run_07,Abs
3,materially,"[[greatly, 0.721524178981781], [remedying, 0.6...",coha,1800,1900,run_07,Abs
4,expected,"[[desired, 0.7255516052246094], [expect, 0.719...",coha,1800,1900,run_07,Abs
...,...,...,...,...,...,...,...
376941,coleworts,"[[persely, 0.9381764531135559], [mulberries, 0...",eebo_tcp,1500,1600,run_02,Conc
376942,confectio,"[[squilliticum, 0.9632958173751831], [syrupus,...",eebo_tcp,1500,1600,run_02,Conc
376943,pane,"[[extendit, 0.6996978521347046], [apprehendit,...",eebo_tcp,1500,1600,run_02,Conc
376944,fries,"[[opprest, 0.8284613490104675], [thisbie, 0.82...",eebo_tcp,1500,1600,run_02,Conc


In [22]:
from tqdm import tqdm

def get_top_words_in_df_slice(period_df,min_count=2):
    _df=pd.DataFrame()
    for word,wdf in tqdm(list(period_df.groupby('word'))):
        # collect nieghborhoods, find meta-top
        nbds = pd.DataFrame(list(wdf.neighborhood.apply(dict)))

        # sort
        top = pd.DataFrame()
        top['count']=nbds.count()
        top['mean']=nbds.mean()
        top=top.sort_values(['count','mean'],ascending=False)
        top['rank']=[i+1 for i in range(len(top))]
        
        # add back
        top=top.rename_axis('word2').reset_index()#.dropna()
        top['word1']=word
        _df=_df.append(top[top['count']>=min_count])
    return _df

In [23]:
newdf=pmap_groups(
    get_top_words_in_df_slice,
    df.groupby(['absconc','period_start']),
    desc='Computing top words per group',
    num_proc=7,
    progress=True
)
newdf

Computing top words per group [x7]: 100%|██████████| 10/10 [16:40<00:00, 100.04s/it]


,word2,count,mean,rank,word1,absconc,period_start
0,burlero,2,0.980984,1,a-la,Conc,1600
1,lilli,2,0.933669,2,a-la,Conc,1600
2,lero,2,0.926497,3,a-la,Conc,1600
3,bullen,2,0.870457,4,a-la,Conc,1600
4,tol,2,0.759921,5,a-la,Conc,1600
...,...,...,...,...,...,...,...
16,paprika,2,0.908866,17,zucchini,Conc,1900
17,tofu,2,0.907589,18,zucchini,Conc,1900
18,broccoli,2,0.904036,19,zucchini,Conc,1900
19,tortillas,2,0.903416,20,zucchini,Conc,1900


In [24]:
newdf.to_csv('../../data/dists/data.top_words.combined.csv')

In [13]:
newdf['count'].value_counts()

2     316541
3     122466
4      62042
5      36578
6      23686
7      16754
10     14575
8      12356
9      10286
Name: count, dtype: int64

In [17]:
words1=set(newdf.word1)
len(words1)

20418

In [9]:
# xdf.tail(100)